# Mounting

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Imports

In [ ]:
!pip install keras==2.4.3
!pip install tensorflow==2.3.0

In [2]:
# First of all we importthe python libraries that we will use

import os

import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Sequential, Model
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.optimizers import Adam
import keras.backend as K


# Plankton network

Change the current directory, so we can access easier to the data sets and save our model in the same folder where the notebook is placed.

In [3]:
os.chdir('/content/drive/My Drive/Colab Notebooks/MLNN/Project3')
print(os.getcwd())
base_dir = 'Data'

# Directory for our training data
train_dir = os.path.join(base_dir,'train')

/content/drive/My Drive/Colab Notebooks/MLNN/Project3


## Network architectures

Here is where the network architecture is designed.
The architecture contains:

1.   Convolutional layer - 32
2.   Pooling of (2,2)
3.   Convolutional layer - 64
4.   Pooling of (2,2)
5.   Convolutional layer - 128
6.   Pooling of (2,2)
7.   Convolutional layer - 128
8.   Pooling of (2,2)
9.   Flatten
10.  Dense of 512
11.  SoftMax

This architecture has the best accuracy.

In [4]:
def network_architecture():
    model = keras.models.Sequential()

    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(keras.layers.MaxPooling2D((2, 2)))

    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))

    model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))

    model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(121, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    
    model.summary()

    return model

The following two networks are attemps to improve the accuracy, but they did not turn out as expected.

In [5]:
def network_architecture_2():
    model = keras.models.Sequential()
    model.add(Conv2D(64, kernel_size=(5, 5),activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(16, kernel_size=(3, 3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(121, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    model.summary()
    return model

In [18]:
def network_architecture_3():
    model = keras.models.Sequential()
    model.add(Conv2D(64, kernel_size=(5, 5),activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(121, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    model.summary()
    return model

## Datasets generation

Here is where the images are preprocessed. Data augmentation is used to create new data from the training data base. The train data is divided in to subsets, one for the training phase and the other for the validation phase. All images are resized to 128x128 to make them of the same shape.

In [7]:
def preprocess_images():
    train_datagen = keras.preprocessing.image.ImageDataGenerator(
        # Rescale images
        rescale=1./255,
        # Rotate images 40º
        rotation_range=40,
        # Width shift images 0.2
        width_shift_range=0.2,
        # Heigth shift images 0.2
        height_shift_range=0.2,
        # Shear image 0.2
        shear_range=0.2,
        # Zoom image 0.2
        zoom_range=0.2,
        # Image can flip horizontally
        horizontal_flip=True,
        # 33% of data image split in validation generator
        validation_split=0.33)
    return train_datagen

In [8]:
def generate_train_set(train_datagen):
    train_generator = train_datagen.flow_from_directory(
        # Train directory
        train_dir,
        # Images resized to 128x128
        target_size=(128, 128),
        # Batch size
        batch_size=20,
        # Since we use categorical_crossentropy loss, we need categorical labels
        class_mode='categorical',
        # This will be the train set
        subset = 'training')
    return train_generator

In [9]:
def generate_validator_set(train_datagen):
    validation_generator = train_datagen.flow_from_directory(
        # Train directory
        train_dir,
        # Images resized to 128x128
        target_size=(128, 128),
        # Batch size
        batch_size=20,
        # Since we use categorical_crossentropy loss, we need categorical labels
        class_mode='categorical',
        # This will be the validation set
        subset = 'validation')
    return validation_generator

## Fit the model

The model needs to get fit with the train set and validate with the validate set.

In [10]:
def model_fit_generator(model,train_generator,validation_generator):
    history = model.fit_generator(
        # Train set generator
        train_generator,
        # Number of steps per epoch
        steps_per_epoch=100,
        # Number of epochs
        epochs=30,
        # Use as validation data the validation set
        validation_data=validation_generator,
        # Number of validation steps
        validation_steps=50)
    return history

## Main function

In [21]:
def train_and_validate_network():

    train_datagen = preprocess_images()

    train_generator = generate_train_set(train_datagen)
    validation_generator = generate_validator_set(train_datagen)

    for data_batch, labels_batch in train_generator:
        print('data batch shape:', data_batch.shape)
        print('labels batch shape:', labels_batch.shape)
        break
    
    model = network_architecture()

    history = model_fit_generator(model,train_generator,validation_generator)

    model.save('plankton2.h5')

    return history

Results for our best network_architecture

In [22]:
plankton_history = train_and_validate_network()

Found 20388 images belonging to 121 classes.
Found 9948 images belonging to 121 classes.
data batch shape: (20, 128, 128, 3)
labels batch shape: (20, 121)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (N

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
100/100 [==============================] - 82s 816ms/step - loss: 4.4378 - acc: 0.0447 - val_loss: 4.2046 - val_acc: 0.0640
Epoch 2/30
100/100 [==============================] - 82s 817ms/step - loss: 4.2555 - acc: 0.0639 - val_loss: 4.1035 - val_acc: 0.1050
Epoch 3/30
100/100 [==============================] - 80s 803ms/step - loss: 4.0194 - acc: 0.1015 - val_loss: 3.9178 - val_acc: 0.1100
Epoch 4/30
100/100 [==============================] - 81s 807ms/step - loss: 3.9004 - acc: 0.0975 - val_loss: 3.8191 - val_acc: 0.1040
Epoch 5/30
100/100 [==============================] - 81s 810ms/step - loss: 3.8373 - acc: 0.1152 - val_loss: 3.7701 - val_acc: 0.1250
Epoch 6/30
100/100 [==============================] - 81s 812ms/step - loss: 3.7481 - acc: 0.1311 - val_loss: 3.6882 - val_acc: 0.1180
Epoch 7/30
100/100 [==============================] - 80s 805ms/step - loss: 3.6520 - acc: 0.1399 - val_loss: 3.5321 - val_acc: 0.1630
Epoch 8/30
100/100 [==============================] - 8

Results for the network_architecture_2

In [20]:
plankton_history = train_and_validate_network()

Found 20388 images belonging to 121 classes.
Found 9948 images belonging to 121 classes.
data batch shape: (20, 128, 128, 3)
labels batch shape: (20, 121)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 124, 124, 64)      4864      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 60, 60, 32)        18464     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (N

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
100/100 [==============================] - 96s 948ms/step - loss: 4.5816 - acc: 0.0489 - val_loss: 4.2988 - val_acc: 0.0670
Epoch 2/30
100/100 [==============================] - 94s 943ms/step - loss: 4.2178 - acc: 0.0606 - val_loss: 4.1751 - val_acc: 0.1000
Epoch 3/30
100/100 [==============================] - 92s 914ms/step - loss: 4.2196 - acc: 0.0649 - val_loss: 4.2289 - val_acc: 0.0530
Epoch 4/30
100/100 [==============================] - 92s 914ms/step - loss: 4.2077 - acc: 0.0697 - val_loss: 4.1558 - val_acc: 0.1140
Epoch 5/30
100/100 [==============================] - 91s 911ms/step - loss: 4.1668 - acc: 0.1002 - val_loss: 4.0135 - val_acc: 0.1210
Epoch 6/30
100/100 [==============================] - 91s 907ms/step - loss: 4.0264 - acc: 0.1117 - val_loss: 3.9558 - val_acc: 0.1130
Epoch 7/30
100/100 [==============================] - 91s 910ms/step - loss: 3.9082 - acc: 0.1220 - val_loss: 3.8084 - val_acc: 0.1260
Epoch 8/30
100/100 [==============================] - 8

Results for the network_architecture_3

In [16]:
plankton_history = train_and_validate_network()

Found 20388 images belonging to 121 classes.
Found 9948 images belonging to 121 classes.
data batch shape: (20, 128, 128, 3)
labels batch shape: (20, 121)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 64)      4864      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (Non

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
100/100 [==============================] - 709s 7s/step - loss: 4.6599 - acc: 0.0330 - val_loss: 4.4635 - val_acc: 0.0640
Epoch 2/30
100/100 [==============================] - 619s 6s/step - loss: 4.2542 - acc: 0.0748 - val_loss: 4.3983 - val_acc: 0.0640
Epoch 3/30
100/100 [==============================] - 543s 5s/step - loss: 4.1134 - acc: 0.0892 - val_loss: 4.3613 - val_acc: 0.1210
Epoch 4/30
100/100 [==============================] - 490s 5s/step - loss: 4.2109 - acc: 0.0745 - val_loss: 4.3049 - val_acc: 0.1110
Epoch 5/30
100/100 [==============================] - 429s 4s/step - loss: 4.1670 - acc: 0.0968 - val_loss: 4.2006 - val_acc: 0.1230
Epoch 6/30
100/100 [==============================] - 395s 4s/step - loss: 4.2384 - acc: 0.0767 - val_loss: 4.2178 - val_acc: 0.0830
Epoch 7/30
100/100 [==============================] - 367s 4s/step - loss: 4.1700 - acc: 0.0848 - val_loss: 4.2278 - val_acc: 0.1120
Epoch 8/30
100/100 [==============================] - 325s 3s/step - 